In [ ]:
#new code


import pandas as pd
import re
import os

# Folder path containing the text files
folder_path = 'text data'

# Initialize variables
data = []

# Regex patterns for extraction
page_pattern = re.compile(r'Page (\d+)')
account_pattern = re.compile(r'(\d+)\s+([\d\.]+)\s+(\w+)\s+(\w+)\s+(yes|no)\s+([\d\.]+)\s+(-?[\d\.]+)')

# Helper function to get the guarantor name (previous non-empty line)
def get_guarantor(lines, index):
    for i in range(index - 1, -1, -1):
        if lines[i].strip() and "Accounts summary" not in lines[i] and "Guarantor" not in lines[i]:
            return lines[i].strip()
    return ""

# Process each text file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)

        # Extract practice name from the file content
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Initialize page number and practice name for each file
        page_number = None
        practice_name = None

        # Parse the text lines
        for i, line in enumerate(lines):
            # Update page number
            page_match = page_pattern.search(line)
            if page_match:
                page_number = int(page_match.group(1))

            # Capture practice name from the line after 'Account Summary By Guarantor'
            if 'Account Summary By Guarantor' in line and i + 1 < len(lines):
                practice_name = lines[i + 1].strip()

            # Extract account details
            account_match = account_pattern.search(line)
            if account_match:
                acct_num, monthly_payment, status, fin_group, industrial, period_balance, balance = account_match.groups()

                # Get the guarantor or leave it blank if no prior entry
                guarantor = get_guarantor(lines, i)

                # Append the extracted data
                data.append([page_number, guarantor, practice_name, acct_num, monthly_payment, status, fin_group, industrial, period_balance, balance])

# Create DataFrame
columns = ["Page No", "Guarantor", "Practice Name", "Acct Num", "Monthly Payment", "Status", "Fin Group", "Industrial", "Period Balance", "Balance"]
df = pd.DataFrame(data, columns=columns)

# Save consolidated DataFrame to CSV
output_path = 'consolidated.csv'
df.to_csv(output_path, index=False)

output_path
